# Purpose
- This file is a scratch-pad for sql queries

In [1]:
%load_ext sql
%sql duckdb:///../data/referral.db --alias referralsDB

c:\Users\MatthewLyn\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
%sql referralsDB

In [3]:
%%sql
CREATE OR REPLACE TABLE coc as

SELECT DISTINCT * FROM read_csv_auto('../data/reference/coc.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
818829


In [4]:
%%sql 
CREATE OR REPLACE TABLE team as 

SELECT DISTINCT * FROM read_csv_auto('../data/reference/user_list.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
266


In [5]:
%sql CREATE OR REPLACE TABLE hcpcs as SELECT DISTINCT * FROM read_csv_auto('../data/reference/cpt_hcpcs.csv', header=True)

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
16991


In [6]:
%sql CREATE OR REPLACE TABLE icd10cm as SELECT DISTINCT * FROM read_csv('../data/reference/icd10cm.csv', header=True, columns={'code_value': 'VARCHAR', 'code_long_description': 'VARCHAR'})

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
95387


In [7]:
%%sql 
CREATE OR REPLACE TABLE cln_referrals as 

SELECT DISTINCT * FROM read_parquet('../data/clean_referrals/*.parquet')

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
100358


- No Auth Required ->	Appointment Scheduled or Member to Schedule or Unable to Contact
- Approved	       ->   Appointment Scheduled or Member to Schedule or Unable to Contact
- Denied	       ->   Open
- Withdrawn	       ->   Open

In [8]:
%%sql 
CREATE OR REPLACE TABLE bi_referrals as

WITH main as (

    SELECT DISTINCT rf.*
    , COALESCE(tm.Team, 'N/A') as User_Team

    , UPPER("Last Name") || UPPER("First Name") as patient_name

    , c.health_plan
    , c.care_partner_name
    , c.care_partner_group

    , UPPER("Last Name") || UPPER("First Name") || "DOB" as patient_id

    , patient_id || coalesce("Plan ID", 'NA') || coalesce("Referring Provider NPI", 'NA') || "Referral Date" || coalesce("Procedure", 'NA') || coalesce(Specialty, 'NA') as Referral_keyid

    , CASE
        WHEN "Approval Status" IN ('No HP Auth Required', 'HP Approved', 'Approved (comments required)', 'Complete/no Auth# needed')  
            AND "Visit Status" IN ('Appointment Scheduled', 'Member to Schedule', 'Unable to Contact', 'Denied', 'Withdrawn')
            THEN 'Completed'
        WHEN "Approval Status" IN ('Denied- Per Insurance Plan', 'Denied- Per Medical Director Review')
            AND "Visit Status" = 'Open'
            THEN 'Completed'
        ELSE "Referral Status" END as "Updated Status"

    , RIGHT("Procedure", 5) as Proc_code
    , pc.code_short_description as Proc_name 

    , FORMAT(
        '({}) {}-{}'
        , substring("Mobile Phone", 1, 3), substring("Mobile Phone", 3, 3), substring("Mobile Phone", 6, 4)
    ) as "Fmt Mobile Phone"

    , FORMAT(
        '({}) {}-{}'
        , substring("Home Phone", 1, 3), substring("Home Phone", 3, 3), substring("Home Phone", 6, 4)
    ) as "Fmt Home Phone"

    , icd.code_long_description AS "Diagnosis Description"

    FROM cln_referrals AS rf

    LEFT JOIN icd10cm AS icd
        ON trim(rf."Diagnosis") = trim(icd.code_value)

    LEFT JOIN hcpcs AS pc
        on RIGHT("Procedure", 5) = pc.code_value

    LEFT JOIN team as tm
        on  "User_FName" = tm.Fname
        AND "User_LName" = tm.Lname

    LEFT JOIN coc as c
        ON rf."Plan ID" = c.member_id
        AND date_part('month', "Referral Date":: DATE) = c.month_ym:: INTEGER
        AND date_part('year', "Referral Date":: DATE) = c.year_ym:: INTEGER

    WHERE "Visit Status" IS NOT NULL
)

, max_updatedt as (
    SELECT DISTINCT
        Referral_keyid
        , Update_DT
        , MAX(Update_DT) OVER (PARTITION BY Referral_keyid) as Last_UpdateDT

    FROM main 
)

SELECT DISTINCT m.* 

FROM main as m

INNER JOIN max_updatedt as dt
    ON m.Update_DT = dt.Last_UpdateDT
    AND m.Referral_keyid = dt.Referral_keyid

*  (referralsDB) duckdb:///../data/referral.db


FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

Done.


Count
96094


In [14]:
%%sql
SELECT DISTINCT 
    REPLACE(REPLACE(REGEXP_REPLACE(UPPER("Health Plan"), '[(*)]', ' ', 'g'), '-', ' '), '  ', ' ')

FROM bi_referrals

*  (referralsDB) duckdb:///../data/referral.db
Done.


"replace(replace(regexp_replace(upper(""Health Plan""), '[(*)]', ' ', 'g'), '-', ' '), ' ', ' ')"
OSCAR INSURANCE COMPANY OF FLORIDA
AMBETTER CENTRUM CONTRACT
AMBETTER
OSCAR HEALTH
UNITED HEALTHCARE EXCHANGE
CIGNA MEDICARE
AMBETTER SUPERIOR HEALTH EXCHANGE
SIMPLY MEDICAID MED CARE
AETNA COMMERCIAL HEALTH
CIGNA EXCHANGE


In [17]:
%%sql

COPY (SELECT DISTINCT REPLACE(REPLACE(REPLACE(REGEXP_REPLACE(UPPER("Health Plan"), '[(*)]', ' ', 'g'), '-', ' '), '  ', ' '), '  ', ' ') FROM bi_referrals) TO '../data/reference/health_plan2.csv' (FORMAT 'csv')

*  (referralsDB) duckdb:///../data/referral.db
Done.


Count
34
